In [2]:
# Import all necessary tools

from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import scipy
import matplotlib.pyplot as plt

from skimage import measure
import pandas as pd

from glob import glob
from tifffile import imread,imsave

from stardist.models import StarDist3D
from pathlib import Path

import sys
sys.path.append('./')
from utils_3Dsegmentation import (  
    patch_segmentation, norm_X,
    Hough_circle_3D, filter_regions_around_specific_voxel,
    sort_out_interfacing_regions,
    show_4d_with_contours, project_colours, project_inside_edge
)

# from src.utils_3Dsegmentation import (  # noqa: E402
#     patch_segmentation,
#     norm_X
# )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Data directory and results directories

folder = 'Embryo1/'
intensity_folder = 'intensity'
segmentation_results = 'STAR-3D_results'
filtered_results = 'STAR-3D_filtered'

if not os.path.isdir(folder + segmentation_results):
    os.makedirs(folder + segmentation_results)
if not os.path.isdir(folder + filtered_results):
    os.makedirs(folder + filtered_results)

## Automatic nuclei segmentation using STAR-3D

In [ ]:
# Set parameters
    
resolution = np.array([2,0.174,0.174]) #in um
# We have found the network to perform optimally when the objects to be detected are about 5 voxels in the lateral direction
# and 60 voxels axially
expected_object_diameter = 30 #um
optimal_resolution = expected_object_diameter/np.array([5.0,60.0,60.0])
downsampling = resolution/optimal_resolution
gamma_factor = 1.0

In [ ]:
# Load intensity files and model

# Load intensity image files
input_files = sorted(glob(folder + intensity_folder + '/*.tif'))[:]

# Load model
model = StarDist3D(None, name='star-3d', basedir= './')

In [ ]:
# Apply STAR-3D to the images
# This might take a very long time. There's a .py version of the segmentation part of the example script in src/ 
# for use on e.g. a cluster.

for file in input_files:
    
    print(file)
    
    # Name the output file (this is normally part of the intensity image file name)
    output_file_name = os.path.splitext(os.path.basename(file))[0][0:-6] + '_label.tif' 

    # Load image    
    intensity = imread(file,is_ome=False)
    
    # Crop intensity image if possible. Applying STAR-3D can take a very long time so it is better to avoid 
    # segmenting empty parts of the image wherever possible. 
    # We have an embryo segmentation method currently in development which could automate this step
    crop = np.array([[0,intensity.shape[0]],[0,intensity.shape[1]],[0,intensity.shape[2]]])
    intensity_cropped = intensity[crop[0,0]:crop[0,1],crop[1,0]:crop[1,1],crop[2,0]:crop[2,1]]
    
    # Normalise, downsample, and gamma correct image
    X_norm = norm_X(intensity_cropped,downsampling,gamma_factor)

    # Apply STAR-3D in patches to avoid running into Tensorflow's tensor size limit
    # Here we calculated the number of patches from the array size, but this method is not very reliable
    # Increase patches if tensorflow returns an error
    patches = 2**(X_norm.size>np.array([6e7,5e7,4e7])).astype(int) + 2**(X_norm.size>np.array([29e7,24e7,19e7])).astype(int) - 1
    print(patches)
    # Set margins where patches overlap to be bigger than the nuclei
    margins = 40/(resolution/downsampling) # we expect the nuclei to be no more than 40 um in diameter
    
    # Apply STAR-3D and resample label map to the original resolution
    Y_pred = scipy.ndimage.zoom(patch_segmentation(X_norm,model,patches,margins),
                                np.asarray(intensity_cropped.shape)/np.asarray(X_norm.shape),order = 0)

    # Separate different regions with the same label 
    # (although it shouldn't happen that two distinct regions have the same label, it's better to make sure)
    Y_pred = measure.label(Y_pred.astype(int), connectivity=2)
    
    # Pad Y_pred to original size (especially if you're planning to use our tracking methods as well)
    Y_pred_full = np.zeros(intensity.shape)
    Y_pred_full[crop[0,0]:crop[0,1],crop[1,0]:crop[1,1],crop[2,0]:crop[2,1]] = Y_pred
    
    # Save unfiltered result
    imsave(folder + segmentation_results + '/' + output_file_name, Y_pred_full.astype('uint16'), imagej=True, 
           resolution=1/resolution[1:3],
           metadata={
               'spacing': resolution[0],
               'unit': 'um',
               'axes': 'ZYX'
           })

## Filtering

In [ ]:
# Image-wide, property-based filtering

source_folder = segmentation_results
output_folder = filtered_results

label_files = sorted(glob(folder + source_folder + '/*.tif'))[:]

for file in label_files:
    
    print(file)
    
    # Name the output file (this is normally part of the original image file name)
    output_file_name = os.path.splitext(os.path.basename(file))[0][0:-4] + '_filtered.tif'
    
    # Load label image
    label_image = imread(file,is_ome=False)
    
    # Calculate the properties that you wish to filter for
    props = pd.DataFrame(measure.regionprops_table(label_image,properties=["label", "area", "euler_number", "moments"]))
    
    # Filter image
    label_image[np.isin(label_image,props['label'][props['area']<100])] = 0
    label_image[np.isin(label_image,props['label'][props['euler_number']<-1])] = 0
    
    # Save filtered image
    imsave(folder + output_folder + '/' + output_file_name, label_image.astype('uint16'), imagej=True, 
           resolution=1/resolution[1:3],
           metadata={
               'spacing': resolution[0],
               'unit': 'um',
               'axes': 'ZYX'
           })

In [ ]:
# Filtering dust around the embryo
# There's often a lot of dust around the embryo which get segmented by STAR-3D
# We have prepared a few functions that might be useful for removing these

###########################################################################################################
###### We have an embryo segmentation method currently in development which could automate this step ######
###### Also, it might be easier to remove the entire tracks of these unwanted structures ##################
###### (see github.com/akarsa/optimal3dtracks) ############################################################
###########################################################################################################

source_folder = filtered_results
output_folder = filtered_results

label_files = sorted(glob(folder + source_folder + '/*.tif'))[:]
intensity_files = sorted(glob(folder + intensity_folder + '/*.tif'))[:]

assert all(Path(input_file_1).name[0:7]==Path(input_file_2).name[0:7] 
           for input_file_1, input_file_2 in zip(intensity_files,label_files))

for file, file_int in zip(label_files,intensity_files):
    
    print(file)
    
    # Name the output file (this is normally part of the original image file name)
    output_file_name = os.path.splitext(os.path.basename(file))[0]
    
    # Load label and intensity images
    label_image = imread(file,is_ome=False)
    intensity = imread(file_int,is_ome=False)
    X_norm = norm_X(intensity,1,1)
    
    # Masking the embryo by fitting a sphere around it using 3D Hough transforms 
    # This actually works pretty well if the embryo is spherical. Unfortunately, this is not always the case.
    hough_radii = np.linspace(25,75,50) # potential embryo radii
    hough_resolution = np.array([2,0.174,0.174]) # resolution of the image
    embryo_boundary = Hough_circle_3D(X_norm, hough_radii, hough_resolution)
    label_image *= embryo_boundary
    
    # Supervised filtering of dust around a specific area of the image
    resolution = np.array([2,0.174,0.174])
    voxel = np.array([0,0,0]) # this will filter around the corner of the image
    radius = 20 # in a 20 um radius
    label_image = filter_regions_around_specific_voxel(label_image, X_norm, resolution, voxel, radius)
    
    # Save filtered image
    imsave(folder + output_folder + '/' + output_file_name, label_image.astype('uint16'), imagej=True, 
           resolution=1/resolution[1:3],
           metadata={
               'spacing': resolution[0],
               'unit': 'um',
               'axes': 'ZYX'
           })

In [ ]:
# Supervised filtering of regions touching across a large surface
# Sometimes STAR-3D creates subsegments within a large nuclei which are interfacing across a large surface
# Here's a function that might be helpful in sorting out these instances.
# It might be easier to do this on the tracks (see github.com/akarsa/optimal3dtracks)

source_folder = filtered_results
output_folder = filtered_results

label_files = sorted(glob(folder + source_folder + '/*.tif'))[:]
intensity_files = sorted(glob(folder + intensity_folder + '/*.tif'))[:]

assert all(Path(input_file_1).name[0:7]==Path(input_file_2).name[0:7] 
           for input_file_1, input_file_2 in zip(intensity_files,label_files))

for file, file_int in zip(label_files,intensity_files):
    
    print(file)
    
    # Name the output file (this is normally part of the original image file name)
    output_file_name = os.path.splitext(os.path.basename(file))[0]
    
    # Load label and intensity images
    label_image = imread(file,is_ome=False)
    intensity = imread(file_int,is_ome=False)
    X_norm = norm_X(intensity,1,1)
    
    # Supervised filtering of regions touching across a large surface
    minimum_percentage_of_connection = 0.05 # below 5% of interfacing area, we assume that they're different nuclei
                                            # supervised filtering is only performed above this threshold
    label_image = sort_out_interfacing_regions(label_image, X_norm, minimum_percentage_of_connection)
    
    # Save filtered image
    imsave(folder + output_folder + '/' + output_file_name, label_image.astype('uint16'), imagej=True, 
           resolution=1/resolution[1:3],
           metadata={
               'spacing': resolution[0],
               'unit': 'um',
               'axes': 'ZYX'
           })

## Display features

In [ ]:
# A python widget for inspecting the segmentation as overlay on the intensity image

label_files = sorted(glob(folder + filtered_results + '/*.tif'))[:]
intensity_files = sorted(glob(folder + intensity_folder + '/*.tif'))[:]

assert all(Path(input_file_1).name[0:7]==Path(input_file_2).name[0:7] 
           for input_file_1, input_file_2 in zip(intensity_files,label_files))

labels = []
intensities = []

for file, file_int in zip(label_files,intensity_files):
    
    print(file)
    
    # Load label and intensity images
    label_image = imread(file,is_ome=False)
    intensity = imread(file_int,is_ome=False)
    X_norm = norm_X(intensity,np.array([1,1,1]),1)
    
    labels.append(np.expand_dims(label_image,axis=0))
    intensities.append(np.expand_dims(X_norm,axis=0))

# Show 4D images with countours
show_4d_with_contours(np.concatenate(intensities),np.concatenate(labels))

In [ ]:
# Show a top-down view of the segmentations

label_image = imread(sorted(glob(folder + filtered_results + '/*.tif'))[0],is_ome=False)

plt.imshow(project_colours(label_image))

In [ ]:
# Colour segmentations according to their position in the morula (from edge (blue) to middle (red))

label_image = imread(sorted(glob(folder + filtered_results + '/*.tif'))[0],is_ome=False)

anisotropy = int(np.round(2/0.174)) # anisotropy of the voxel size
distance_map, distance_map_masked = distance_from_edge_colouring(label_image, anisotropy)

plt.imshow(project_inside_edge(distance_map_masked))